# Custom Forces - Umbrella Sampling

[![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openmm/openmm_workshop_july2023/blob/main/section_2/custom_forces.ipynb)


This notebook introduces the custom forces available in OpenMM. First we apply them to a Lennard-Jones toy system. Then cover an in-depth example of implementing Umbrella Sampling in OpenMM using custom forces.

## Table of Contents
- Setting up environment
- Custom Forces
    - Creating a toy system
    - Custom non-bonded force
    - Custom external force
- Umbrella Sampling
    - System
    - Setting up the windows with SMD
    - Running the windows
    - Analysis
- Extra exercises
- References
- Solutions

## Setting up environment
<a id="env"></a>

In [ ]:
# Execute this cell to install mamba in the Colab environment and then install openmm

if 'google.colab' in str(get_ipython()):
  print('Running on colab')
  !pip install -q condacolab
  import condacolab
  condacolab.install_mambaforge()
else:
  print('Not running on colab.')
  print('Make sure you create and activate a new conda environment!')


**Note:** During this step on Colab the kernel will be restarted. This will produce the error message:
"Your session crashed for an unknown reason. " This is normal and you can safely ignore it.

**Note:** Installing the packages will take several minutes!

In [ ]:

# install openmm
!mamba install -y -c conda-forge openmm  


## Custom Forces
<a id="customforces"></a>

OpenMM provides [Custom Force](http://docs.openmm.org/latest/userguide/theory/03_custom_forces.html) classes. These classes provide great flexibility by allowing the user to specify arbitrary algebraic expressions. We will apply a custom non-bonded force and a custom external force to a Lennard-Jones gas toy system.


### Create toy system
<a id="toysystem"></a>

We will use the OpenMM API to create a LJ system. Note that there are a variety of different test systems available in the OpenMM tools package: https://github.com/choderalab/openmmtools. For our test system we will generate of 3D grid of 1000 Lennard Jones particles using parameters for Argon.

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import numpy as np
import itertools
from sys import stdout

# number of atoms
N=1000

# box length
L=5.0*unit.nanometer

# use values for Argon from openmm-tools test systems.
sigma = 3.4 * unit.angstrom
epsilon = 0.238 * unit.kilocalories_per_mole

# Make an empty topology
topology = app.Topology()

# add a single chain
chain = topology.addChain()

# add each atom
for i in range(N):
    residue = topology.addResidue(name='Ar', chain=chain)
    atom = topology.addAtom(name='Ar', element=app.element.get_by_symbol('Ar'), residue=residue)

topology.setPeriodicBoxVectors(np.eye(3)*L)
# check the topology
print(topology)

# generate positions on a grid
M=int(np.cbrt(N))
dx = L._value/M
print(dx)
x = np.linspace(0,L._value-dx,int(np.cbrt(N)))
positions = np.array(list(itertools.product(x,x,x)))*unit.nanometers

# create the system and add the particles to it
system = mm.System()
system.setDefaultPeriodicBoxVectors(*topology.getPeriodicBoxVectors())
for atom in topology.atoms():
    system.addParticle(atom.element.mass)

# output the topology
with open('topology_lj.pdb', 'w') as f:
    app.PDBFile.writeFile(topology, positions, f)


The initial topology looks like this:
![topology_lj](images/topology_lj.png)

**Note**: This image was generated using Ovito https://www.ovito.org/ an alternative molecular visulization software.


We now have a `Topology` and a `System` but no forces defined yet.


### Custom non-bonded force
<a id="customnonbonded"></a>

We will create a [`customNonBondedForce`](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.CustomNonbondedForce.html) that for simplicity uses a Lennard-Jones interaction. When you create a custom force you give it the potential energy expression as a string. The custom force class then computes the gradient of this energy expression to compute the forces.

In [ ]:
custom_nb_force = mm.CustomNonbondedForce('4*epsilon*((sigma/r)^12-(sigma/r)^6); sigma=0.5*(sigma1+sigma2); epsilon=sqrt(epsilon1*epsilon2)')

The potential energy depends on two parameters, `sigma` and `epsilon`, and we have defined the combination rules to calculate them. 
We need to tell the customForce that these are `perParticleParameters` that is each particle will have its own value for them.

In [ ]:
custom_nb_force.addPerParticleParameter('sigma')
custom_nb_force.addPerParticleParameter('epsilon')

We then need to set the values of sigma and epsilon for each particle. This is done using the `addParticle()` method.

In [ ]:
for i in range(N):
    custom_nb_force.addParticle([sigma, epsilon])

Set the cutoff method settings.

In [ ]:
custom_nb_force.setNonbondedMethod(mm.CustomNonbondedForce.CutoffPeriodic)
custom_nb_force.setCutoffDistance(3.0*sigma)

We can now add the force to the system.

In [ ]:
system.addForce(custom_nb_force)

The system is now ready to simulate.

In [ ]:
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)

simulation.reporters.append(app.PDBReporter("lj_traj.pdb", 100))

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, speed=True))

simulation.step(1000)

We can compare the speed to the standard nonbonded force.

In [ ]:

standard_system = mm.System()
standard_system.setDefaultPeriodicBoxVectors(*topology.getPeriodicBoxVectors())
for atom in topology.atoms():
    standard_system.addParticle(atom.element.mass)

nb_force = mm.NonbondedForce()
nb_force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
nb_force.setCutoffDistance(3.0*sigma)
charge = 0.0

for i in range(N):
    nb_force.addParticle(charge, sigma, epsilon)

standard_system.addForce(nb_force)

integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds)
standard_simulation = app.Simulation(topology, standard_system, integrator)
standard_simulation.context.setPositions(positions)

standard_simulation.reporters.append(app.PDBReporter("lj_traj.pdb", 100))

standard_simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, speed=True))

standard_simulation.step(1000)

Depending on the hardware you will get different speeds. We expect that standard NonBondedForce will be faster but typically not more than double the speed.

The functional form of the custom nonbonded force can be as complex as you need.


### Custom external force
<a id="customexternal"></a>

Another very flexible custom force is [`CustomExernalForce`](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.CustomExternalForce.html#openmm.openmm.CustomExternalForce). This class implements an 'external' force on particles. We can create a `CustomExternalForce` that attracts each particle to a target position (x0) in the x direction via a harmonic potential. This force depends on a global parameter k and a perParticle parameter x0.

**Exercise 1.** You will need to add the per particle parameter 'x0' to the force.

In [ ]:
external_force = mm.CustomExternalForce('k*periodicdistance(x,0,0,x0,0,0)^2')
external_force.addGlobalParameter('k', 10.0)

# add the per particle parameter 'x0' to the force
FIXME

# add each particle to the external_force
# for this custom force we give the index of each particle to add,
# and then a list of the perParticleParameters
for i in range(N):
    external_force.addParticle(i, [0.0])


We have now defined a force that pulls all particles to the x origin. We can add this force to a system and run a simulation.

**Exercise 2**. You will need to the external force to the system.

In [ ]:
from copy import deepcopy

new_system = mm.System()
new_system.setDefaultPeriodicBoxVectors(*topology.getPeriodicBoxVectors())
for atom in topology.atoms():
    new_system.addParticle(atom.element.mass)


new_system.addForce(deepcopy(nb_force))

# add the external_force to the system
FIXME


integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds)
new_simulation = app.Simulation(topology, new_system, integrator)
new_simulation.context.setPositions(positions)

new_simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, speed=True))

new_simulation.reporters.append(app.PDBReporter("lj_traj.pdb", 100))

new_simulation.step(2000)

**Exercise 3.** Visualize the trajectory file `'lj_traj.pdb'` to check the external force is working correctly. Now change the expression to `'k*(x-x0)^2'` and rerun the simulation. What happens to the trajectory? Why is this?

## Umbrella sampling
<a id="umbrellasampling"></a>

We will now use the custom forces to implement umbrella sampling in OpenMM.
 
Umbrella sampling is a technique used in molecular dynamics simulations to improve sampling in systems containing high barriers in the free energy landscape [[1,2]](#references). With plain MD simulations the system can remain trapped in free energy minima whereas umbrella sampling adds extra biasing potentials to drive the system in the direction of a chosen collective variable (CV). This enables the calculation of free energy profiles much more efficiently than is possible with regular MD alone. 


Figure 1 shows an overview of umbrella sampling. For more explanation of the theory of Umbrella sampling see [umbrella_sampling_theory.md](umbrella_sampling_theory.md).


![umbrellasampling](./images/umbrella_sampling.svg)

**Figure 1. Umbrella sampling method to compute a free energy profile.** (a) Multiple biasing potentials are placed across the collective variable $x$. The blue curve is the free energy of the system which we are trying to calculate. (b) Simulations are run for each window $w$. The resulting biased probability distributions $P'(x)$ are plotted. (c) The unbiased free energies $F_i$ from each window. They are each offset by a different $C_i$. (d) The Weighted Histogram Analysis Method (WHAM) is used to combine the windows and compute the free energy curve.

### Umbrella sampling simulations
 
Most umbrella sampling simulations have three main steps:
 
1. Preparing the windows. This is usually done using Steered Molecular Dynamics. 
2. Running the windows. These can be computed in parallel to take advantage of compute resources.
3. Analysing the results. Typically this involves computing a PMF with WHAM. 
 
Often you will find in step 3 that the simulations do not produce a nicely converged PMF. You will need to return to step 1 and change some settings. This trial and improvement feedback loop is a normal part of the process. For this tutorial we will use settings already known to work. 

### System
<a id="umbrellasystem"></a>
 
This tutorial will use umbrella sampling to compute the free energy profile of the end-to-end distance ($r$) of deca-alanine in vacuum. Deca-alanine is commonly used as a toy system [[3](#References)]. Its equilibrium structure is a stable alpha-helix. Starting with the alpha-helix structure we will first perform a SMD simulation to pull it from a helix into a coil. Next, we will run 24 umbrella sampling windows in the range 1.3nm to 3.3nm. Finally, we will compute the PMF along $r$. Figure 2 shows the initial alpha-helix structure and the final extended coil structure.

![deca-alanine](./images/deca-alanine.png)

**Figure 2** Structure of deca-alanine for end-to-end distance $r$=1.3 and $r$=3.3nm.


In [ ]:
# get the deca-alanine PDB file
!wget https://raw.githubusercontent.com/openmm/openmm_workshop_july2023/main/section_2/deca-ala.pdb


### Step 1 - Setting up the windows with SMD
<a id="setupwindows"></a> 

The script below does the following steps:
-   Loads in a PDB file.
-   Defines a collective variable between the first and last alpha-carbon.
    It does this using [`CustomBondForce`](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.CustomBondForce.html#openmm.openmm.CustomBondForce) to record the distance between the two atoms.
-   Adds a harmonic restraint to the CV using [`CustomCVForce`](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.CustomCVForce.html#openmm.openmm.CustomCVForce).
-   Runs a simulation where the location of the harmonic restraint is moved with constant velocity from 1.3nm to 3.3nm (This is called constant velocity steered MD).
-   Saves a configuration for each of the 24 equally spaced windows between 1.3nm and 3.3nm.


You should read through the full script before you run the cell!

**Exercise 4**. You will need to add code to the cell below to complete the setting up of the `CustomCVForce`.

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout
import numpy as np

pdb = app.PDBFile('deca-ala.pdb')

forcefield = app.ForceField('amber14-all.xml')

# We have a single molecule in vacuum so we use no cutoff.
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds, hydrogenMass=1.5*unit.amu)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.reporters.append(app.DCDReporter('smd_traj.dcd', 10000))
simulation.reporters.append(app.StateDataReporter(stdout, 10000, step=True, time=True, potentialEnergy=True, temperature=True, speed=True))

# equilibrate
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.step(1000)

# Define the CV as the distance between the CAs of the two end residues.
# You can open the "deca-ala.pdb" file to check these indices.
index1 = 8
index2 = 98
cv = mm.CustomBondForce('r')
cv.addBond(index1, index2)

# now setup SMD

# starting value
r0 = 1.3*unit.nanometers

# force constant
fc_pull = 1000.0*unit.kilojoules_per_mole/unit.nanometers**2 

# pulling speed
v_pulling = 0.02*unit.nanometers/unit.picosecond # nm/ps

# simulation time step
dt = simulation.integrator.getStepSize()

# total number of steps
total_steps = 30000 # 120ps

# number of steps to run between incrementing r0 (1 makes the simulation slow)
increment_steps = 10

# define a harmonic restraint on the CV
# the location of the restrain will be moved as we run the simulation
# this is constant velocity steered MD
pullingForce = mm.CustomCVForce('0.5 * fc_pull * (cv-r0)^2')

# Add the global parameters fc_pull and r0 to the CustomCVforce
FIXME

pullingForce.addCollectiveVariable("cv", cv)
system.addForce(pullingForce)
simulation.context.reinitialize(preserveState=True)

# define the windows
# during the pulling loop we will save specific configurations corresponding to the windows
windows = np.linspace(1.3, 3.3, 24)
window_coords = []
window_index = 0

# SMD pulling loop
for i in range(total_steps//increment_steps):
    simulation.step(increment_steps)
    current_cv_value = pullingForce.getCollectiveVariableValues(simulation.context)
    
    if (i*increment_steps)%5000 == 0:
        print("r0 = ", r0, "r = ", current_cv_value)
    
    # increment the location of the CV based on the pulling velocity
    r0 += v_pulling * dt * increment_steps
    simulation.context.setParameter('r0',r0)

    # check if we should save this config as a window starting structure
    if (window_index < len(windows) and current_cv_value >= windows[window_index]):
        window_coords.append(simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions())
        window_index += 1

# save the window structures
for i, coords in enumerate(window_coords):
    outfile = open(f'window_{i}.pdb', 'w')
    app.PDBFile.writeFile(simulation.topology,coords, outfile)
    outfile.close()

Once the script has completed running there will be 24 new pdb files called "window_n.pdb" where n is an integer from 0 to 23.
 
We now have the initial configurations for the umbrella sampling windows.

### Step 2 - Running the windows
<a id="runningwindows"></a>
 
The script to run the windows is very similar to the script in step 1. The key differences are that we load in an initial structure that corresponds to each specific window and that the harmonic restraint on the CV does not move. The script below defines a function to run one window. It re-uses the `Simulation` we created in step 1.

**Exercise 5**. You will need to add the code to set the value of r0 for the window.

In [ ]:
def run_window(window_index):

    print('running window', window_index)
    
    # load in the starting configuration for this window
    pdb = app.PDBFile(f'window_{window_index}.pdb')

    # we can reuse the existing Simulation
    simulation.context.setPositions(pdb.positions)

    # set the fixed location of the harmonic restraint for this window
    r0 = windows[window_index]

    # set the value of r0 in the simulation
    FIXME

    # run short equilibration with new positions and r0
    simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
    simulation.step(1000)

    # run the data collection

    # total number of steps
    total_steps = 10000 # 400 ps

    # frequency to record the current CV value
    record_steps = 100

    # run the simulation and record the value of the CV.
    cv_values=[]
    for i in range(total_steps//record_steps):
        simulation.step(record_steps)

        # get the current value of the cv
        current_cv_value = pullingForce.getCollectiveVariableValues(simulation.context)
        cv_values.append([i, current_cv_value[0]])

    # save the CV timeseries to a file so we can postprocess 
    np.savetxt(f'cv_values_window_{window_index}.txt', np.array(cv_values))

    print('Completed window', window_index)

We then run all 24 windows

In [ ]:
for n in range(24):
    run_window(n)

Once all the window simulations have completed you will have the CV timeseries files: "cv_values_window_n.txt"

### Step 3 - Analysis - compute the PMF
<a id="computepmf"></a>

The first thing to check is that the histograms of the CV timeseries have good overlap. Here is an example:
![histogram](./images/hist.png)


You can plot yours with the script below
(This script also produces the metadata file we will need for the next step).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the histograms
metafilelines = []
for i in range(len(windows)):
    data = np.loadtxt(f'cv_values_window_{i}.txt')
    plt.hist(data[:,1])
    metafileline = f'cv_values_window_{i}.txt {windows[i]} 1000\n'
    metafilelines.append(metafileline)

plt.xlabel("r (nm)")
plt.ylabel("count")

with open("metafile.txt", "w") as f:
    f.writelines(metafilelines)

To compute the PMF we can use WHAM [[1](#References)]. An easy to use and widely compatible implementation is the WHAM program by Alan Grossfield which can be downloaded here: http://membrane.urmc.rochester.edu/?page_id=126. It is a C program so it will need to be compiled. The command below should work on Linux and Mac.

**Note:** If the next two cells don't work for you then skip them and move onto the plotting phase with the file we have provided for you --- "pmf_backup.txt" which you can use instead of "pmf.txt".

In [ ]:
!wget http://membrane.urmc.rochester.edu/sites/default/files/wham/wham-release-2.0.11.tgz
!tar xf wham-release-2.0.11.tgz
!cd wham/wham && make

To use `wham` we need a metadata file that lists the names of each CV timeseries file, the location of the harmonic restraints, and the value of the spring constant. We created this in the histogram plotting script earlier.
 
The `wham` program is run using command line arguments. Read the documentation to find out more: http://membrane.urmc.rochester.edu/sites/default/files/wham/doc.pdf
 
The command below will compute the PMF from our data. The command line arguments correspond to a range of 1.3nm to 3.3nm, 50 histogram bins, a tolerance of 1e-6, and a temperature of 300K.


In [ ]:
!./wham/wham/wham 1.3 3.3 50 1e-6 300 0 metafile.txt pmf.txt > wham_log.txt

We can then plot the computed PMF. It should look something like this:

![pmf.png](./images/pmf.png)

In [ ]:
# plot the PMF
pmf = np.loadtxt("pmf.txt")
plt.plot(pmf[:,0], pmf[:,1])
plt.xlabel("r (nm)")
plt.ylabel("PMF (kJ/mol)")
plt.show()

## Extra exercises
<a id="extra"></a>

- In the [umbrella sampling setup stage](#setupwindows) we used constant velocity steered-MD to pull the peptide apart. There is also constant force steered MD. This is where a constant force is applied to the system to pull it into the desired configuration. Modify the window setup script to perform constant force steered MD. Tip: You can use `CustomExternalForce`. 
 
- Run the full umbrella sampling procedure for the deca-alanine peptide solvated in a water box.


## References
<a id="References"></a>

[1] Kumar, S., Rosenberg, J.M., Bouzida, D., Swendsen, R.H. and Kollman, P.A. (1992), *The weighted histogram analysis method for free-energy calculations on biomolecules. I. The method.* J. Comput. Chem., 13: 1011-1021. https://doi.org/10.1002/jcc.540130812  
[2] Kästner, J. (2011), *Umbrella sampling.* WIREs Comput Mol Sci, 1: 932-942. https://doi.org/10.1002/wcms.66  
[3] Park, S., Khalili-Araghi, F., Tajkhorshid, E., and Schulten, K. (2003), *Free energy calculation from steered molecular dynamics simulations using Jarzynski’s equality.* J. Chem. Phys., 119 (6): 3559–3566. https://doi.org/10.1063/1.1590311  

## Solutions
<a id="solutions"></a>

*Exercise 1.* Add the per particle parameter:
```python
external_force.addPerParticleParameter('x0')
```

*Exercise 2.* Add the external force:
```python
new_system.addForce(external_force)
```

*Exercise 3.* If you visualize the trajectory you will see all the atoms get pulled to x=0 as show in the image on the left below. If you use `(x-x0)^2` instead of  `periodicdistance()` you will see that the custom force does not work as expected due to the PBCs not being taken into account (shown in the right image below). 

![solution 3](images/solution3.png)

*Exercise 4.* Add the global parameters fc_pull and r0 to the CustomCVforce:
```python
pullingForce.addGlobalParameter('fc_pull', fc_pull)
pullingForce.addGlobalParameter('r0', r0)
```

*Exercise 5.* 
```python
simulation.context.setParameter('r0',r0)
```
